In [85]:
try:
    import mesa
except ImportError:
    !pip install mesa
    import mesa

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Resource Classes

In [86]:
class Sugar(mesa.Agent):

    def __init__(self, model, pos, max_sugar):
        super().__init__(model)
        #self.grid.place_agent(sugar, pos)
        #self.pos=pos
        self.amount = max_sugar
        self.max_sugar = max_sugar

    def step(self):

        self.amount = min(self.amount + 1, self.max_sugar)
        # print("unique id :", self.unique_id, "amount: ", self.amount, "max :", self.max_sugar)



In [87]:
class Spice(mesa.Agent):

    def __init__(self, model, pos, max_spice):
        super().__init__(model)
        self.amount = max_spice
        self.max_spice = max_spice


    def step(self):

        self.amount = min(self.amount + 1, self.max_spice)
        # print("unique id :", self.unique_id, "amount: ", self.amount, "max :", self.max_spice)


In [88]:
class Trader(mesa.Agent):
    def __init__(self, model, pos, moore=False, sugar=0, spice=0, metabolism_sugar=0, metabolism_spice=0, vision=0):

        super().__init__(model)
        self.moore = moore
        self.sugar = sugar
        self.spice = spice
        self.metabolism_sugar = metabolism_sugar
        self.metabolism_spice = metabolism_spice
        self.vision = vision




# model class

In [89]:
class SugarscapeG1mt(mesa.Model):
    def __init__(self, width=50, height=50, initial_population=200,
                 endowment_min=20, endowment_max=50,
                 metabolism_min=1, metabolism_max=5,
                 vision_min=1, vision_max=5):
        
        # 부모 클래스(mesa.Model) 초기화
        super().__init__()


        self.width = width
        self.height = height

        # Initialize
        self.initial_population = initial_population
        self.endowment_min = endowment_min
        self.endowment_max = endowment_max
        self.metabolism_min = metabolism_min
        self.metabolism_max = metabolism_max
        self.vision_min = vision_min
        self.vision_max = vision_max


       # torus=True로 하면 팩맨처럼 맵이 연결됩니다.
        self.grid = mesa.space.MultiGrid(self.width, self.height, torus=True)

        self.sugar_distribution = np.genfromtxt("sugar-map.txt")
        self.spice_distribution = np.flip(self.sugar_distribution, axis=0)


        for _, (x, y) in self.grid.coord_iter():
            max_sugar = self.sugar_distribution[x, y]
            if max_sugar > 0:
                sugar = Sugar(self, (x,y), max_sugar)
                self.grid.place_agent(sugar, (x, y))
                # self.schedule.add(sugar) # Mesa 3.0+에서는 필요 없으므로 삭제

            max_spice = self.spice_distribution[x, y]
            if max_spice > 0:
                spice = Spice(self, (x,y), max_spice)
                self.grid.place_agent(spice, (x, y))
                # self.schedule.add(spice) # Mesa 3.0+에서는 필요 없으므로 삭제

        for i in range(self.initial_population):
            x = self.random.randrange(self.width)
            y = self.random.randrange(self.height)

            sugar = self.random.randrange(self.endowment_min, self.endowment_max + 1)
            spice = self.random.randrange(self.endowment_min, self.endowment_max + 1)

            metabolism_sugar = self.random.randrange(self.metabolism_min, self.metabolism_max + 1)
            metabolism_spice = self.random.randrange(self.metabolism_min, self.metabolism_max + 1)

            vision = self.random.randrange(self.vision_min, self.vision_max + 1)

            # Initiate individual agent
            trader = Trader(self, (x,y), moore=False, # moore는 키워드 인자로 유지
                        sugar=sugar, spice=spice, # 나머지 인자들도 키워드 인자로 명시
                        metabolism_sugar=metabolism_sugar, metabolism_spice=metabolism_spice,
                        vision=vision)
            # Place agent
            self.grid.place_agent(trader, (x,y))
            # self.schedule.add(trader) # Mesa 3.0+에서는 필요 없으므로 삭제
            print(trader.unique_id, trader.pos, trader.sugar, trader.spice, trader.metabolism_sugar,trader.metabolism_spice, trader.vision)

    def make_map(self):

        # 도화지를 가로로 길게(12x6 크기) 준비합니다.
        plt.figure(figsize=(12, 6))

        # 첫 번째 그림: 설탕 지도 (1행 2열 중 1번째)
        plt.subplot(1, 2, 1)
        plt.imshow(self.sugar_distribution, origin="lower", cmap="Oranges")
        plt.title("Sugar Map")
        plt.colorbar()

        # 두 번째 그림: 향신료 지도 (1행 2열 중 2번째)
        plt.subplot(1, 2, 2)
        plt.imshow(self.spice_distribution, origin="lower", cmap="BuPu")
        plt.title("Spice Map")
        plt.colorbar()


    def step(self):
        # 모델의 모든 에이전트(Sugar, Spice, Trader)의 step 메서드를 호출합니다.
        # Mesa 3.0+에서는 self.agents를 통해 에이전트들을 관리합니다.
        self.agents.do("step")

    #    for sugar in self.schedule.agents_by_type[Sugar].values():
    #        sugar.step()

    #    self.schedule.steps += 1 #important for data collector to track number of steps

    def run_model(self,step_count=1000):
        for i in range(step_count):
            print("step: ", i)
            self.step()


# Run Model


In [90]:
model = SugarscapeG1mt()
model.run_model(5)



4139 (36, 11) 27 41 5 3 1
4140 (43, 7) 35 21 5 4 5
4141 (22, 32) 20 45 2 1 1
4142 (24, 6) 45 46 4 5 4
4143 (33, 10) 40 41 1 1 2
4144 (7, 33) 47 24 1 5 2
4145 (33, 17) 40 50 2 1 2
4146 (38, 37) 39 31 2 3 5
4147 (42, 10) 36 24 1 1 1
4148 (24, 30) 44 38 3 4 5
4149 (3, 3) 34 33 1 2 2
4150 (25, 2) 29 32 3 1 1
4151 (25, 35) 30 39 1 5 2
4152 (16, 48) 45 26 2 4 1
4153 (29, 14) 44 38 2 1 5
4154 (7, 28) 30 24 5 5 4
4155 (20, 6) 23 45 4 2 4
4156 (0, 19) 29 20 5 3 3
4157 (2, 46) 50 27 4 5 3
4158 (17, 2) 38 39 4 3 1
4159 (2, 24) 41 43 1 2 1
4160 (21, 19) 26 32 5 5 3
4161 (29, 46) 43 43 4 4 4
4162 (36, 21) 48 36 4 2 1
4163 (1, 3) 26 47 1 1 2
4164 (16, 16) 43 37 3 3 5
4165 (38, 16) 42 37 2 1 5
4166 (18, 19) 47 28 2 3 3
4167 (0, 38) 45 35 2 1 4
4168 (0, 9) 45 24 1 4 2
4169 (27, 2) 45 43 2 1 5
4170 (27, 14) 39 33 1 1 1
4171 (23, 7) 37 30 4 3 3
4172 (37, 44) 35 49 4 3 1
4173 (38, 32) 44 31 3 4 3
4174 (25, 25) 44 34 1 3 2
4175 (2, 8) 48 33 4 3 3
4176 (43, 24) 50 35 5 4 1
4177 (47, 5) 49 36 4 2 4
4178 (1,